## Mumbai Segmentation

In [1]:
#import libraries
!conda install beautifulsoup4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import csv
import json 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         165 KB

The following packages will be UPDATED:

    beautifulsoup4: 4.7.1-py36_1 --> 4.9.0-py36_0


beautifulsoup4-4.9.0 | 165 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6  

In [176]:
pd.set_option('max_colwidth', 800) #for better view
# Get the text data od neighbours in mumbai from wikipedia
source = requests.get('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai').text 
soup = BeautifulSoup(source, 'lxml')
print(soup.find('li'))

<li class="toclevel-1 tocsection-1"><a href="#Western_Suburbs"><span class="tocnumber">1</span> <span class="toctext">Western Suburbs</span></a>
<ul>
<li class="toclevel-2 tocsection-2"><a href="#Andheri"><span class="tocnumber">1.1</span> <span class="toctext">Andheri</span></a></li>
<li class="toclevel-2 tocsection-3"><a href="#Mira-Bhayandar"><span class="tocnumber">1.2</span> <span class="toctext">Mira-Bhayandar</span></a></li>
<li class="toclevel-2 tocsection-4"><a href="#Bandra"><span class="tocnumber">1.3</span> <span class="toctext">Bandra</span></a></li>
<li class="toclevel-2 tocsection-5"><a href="#Borivali"><span class="tocnumber">1.4</span> <span class="toctext">Borivali</span></a></li>
<li class="toclevel-2 tocsection-6"><a href="#Dahisar"><span class="tocnumber">1.5</span> <span class="toctext">Dahisar</span></a></li>
<li class="toclevel-2 tocsection-7"><a href="#Goregaon"><span class="tocnumber">1.6</span> <span class="toctext">Goregaon</span></a></li>
<li class="tocleve

In [177]:
table_post = soup.find('li')
mumareas = table_post.find_all('span')
neighbourhood = []

for i in range(0, len(mumareas),2):
    neighbourhood.append(mumareas[i+1].text.strip())
    
        
df_mumbai= pd.DataFrame(data=[neighbourhood]).transpose()
df_mumbai.columns = ['Neighbourhood']
print(df_mumbai)


      Neighbourhood
0   Western Suburbs
1           Andheri
2    Mira-Bhayandar
3            Bandra
4          Borivali
5           Dahisar
6          Goregaon
7        Jogeshwari
8              Juhu
9    Kandivali west
10   Kandivali east
11             Khar
12            Malad
13        Santacruz
14            Vasai
15            Virar
16       Vile Parle


In [145]:
df_mumbai.shape

(17, 1)

#### Cleaning data

In [178]:
#df_mumbai.drop(0,inplace=True)
df_mumbai.head()

,Neighbourhood
1,Andheri
2,Mira-Bhayandar
3,Bandra
4,Borivali
5,Dahisar


In [179]:
#to find mumbai coordinates
address = 'Mumbai, India'

    
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai City are 18.9387711, 72.8353355.


In [180]:
#to find each area coordinates
from geopy.geocoders import Nominatim
latitudes = [] # Initializing the latitude array
longitudes = [] # Initializing the longitude array
for area in df_mumbai["Neighbourhood"] : 
    place_name = area + ",Mumbai, India"
    geolocator = Nominatim(user_agent="googleapis")
    location = geolocator.geocode(area)
    latitudes.append(location.latitude)
    longitudes.append(location.longitude) 

In [181]:
df_mumbai['Latitude'] = latitudes #append the data to dataframe
df_mumbai['Longitude'] = longitudes

In [182]:
df_mumbai.head()

,Neighbourhood,Latitude,Longitude
1,Andheri,19.119698,72.846420
2,Mira-Bhayandar,19.295543,72.857359
3,Bandra,26.049014,85.622828
4,Borivali,19.229068,72.857363
5,Dahisar,19.249450,72.859621


In [183]:
# The code was removed by Watson Studio for sharing.

In [184]:
mum_lat = 18.9387711 
mum_lng = 72.8353355

# Creates map of Mumbai using latitude and longitude values
map_mumbai = folium.Map(location=[mum_lat, mum_lng], zoom_start=10)

# Add markers to map
for lat, lng, neighbourhood in zip(df_mumbai['Latitude'], df_mumbai['Longitude'], df_mumbai['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

In [185]:
# Gets the name of the category

def get_category_type(row):
    categories_list = row['Category']  
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [186]:
explore_df_list = []

for i, nbd_name in enumerate(df_mumbai['Neighbourhood']):  
    
    try :
        ### Getting the data of neighbourhood
        nbd_name = df_mumbai.loc[i, 'Neighbourhood']
        nbd_lat = df_mumbai.loc[i, 'Latitude']
        nbd_lng = df_mumbai.loc[i, 'Longitude']

        radius = 1000 # Setting the radius as 1000 metres
        LIMIT = 30 # Getting the top 30 venues

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={} \
        &client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nbd_lat, nbd_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flattens JSON

        # Filtering the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Renaming the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Gets the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)    
        
        # Gets the data required
        for i, name in enumerate(nearby['Name']):
            s_list = nearby.loc[i, :].values.tolist()  # Converts the numpy array to a python list
            f_list = [nbd_name, nbd_lat, nbd_lng] + s_list
            explore_df_list.append(f_list)
    
    except Exception as e:
        pass

In [187]:
df_venue = pd.DataFrame([item for item in explore_df_list])
df_venue.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']
df_venue.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,Andheri,19.119698,72.84642,Merwans Cake shop,Bakery,19.119300,72.845418
1,Andheri,19.119698,72.84642,Narayan Sandwich,Sandwich Place,19.121398,72.850270
2,Andheri,19.119698,72.84642,Radha Krishna Veg Restaurant,Indian Restaurant,19.115130,72.843060
3,Andheri,19.119698,72.84642,Shawarma Factory,Falafel Restaurant,19.124591,72.840398
4,Andheri,19.119698,72.84642,Temptations,Ice Cream Shop,19.113767,72.841337


In [188]:
# One hot encoding
mumbai_onehot = pd.get_dummies(df_venue[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
mumbai_onehot['Neighbourhood'] = df_venue['Neighbourhood'] 

# Move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + mumbai_onehot.columns[:-1].values.tolist()
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Neighbourhood,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Bookstore,Breakfast Spot,...,Smoke Shop,Snack Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Tea Room,Theater,Train Station,Vegetarian / Vegan Restaurant
0,Andheri,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Andheri,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [189]:
mumbai_grouped = mumbai_onehot.groupby('Neighbourhood').mean().reset_index()
mumbai_grouped.head()

,Neighbourhood,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Bookstore,Breakfast Spot,...,Smoke Shop,Snack Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Tea Room,Theater,Train Station,Vegetarian / Vegan Restaurant
0,Andheri,0.0,0.000000,0.000000,0.033333,0.066667,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.033333
1,Borivali,0.0,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.033333,0.000000,0.033333
2,Dahisar,0.0,0.041667,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.041667,0.000000
3,Goregaon,0.0,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.033333,0.000000,...,0.033333,0.033333,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.033333
4,Jogeshwari,0.0,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.062500,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [190]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [191]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = mumbai_grouped['Neighbourhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Andheri,Indian Restaurant,Fast Food Restaurant,Bakery,Ice Cream Shop,Café,Sandwich Place,Vegetarian / Vegan Restaurant,Food Court,Department Store,Pizza Place
1,Borivali,Ice Cream Shop,Chinese Restaurant,Food Truck,Indian Restaurant,Clothing Store,Sandwich Place,Gym,Gym / Fitness Center,Fast Food Restaurant,Juice Bar
2,Dahisar,Indian Restaurant,Chinese Restaurant,Restaurant,Juice Bar,Café,Bar,Seafood Restaurant,Diner,Gym,Hotel Bar
3,Goregaon,Fast Food Restaurant,Indian Restaurant,Gym / Fitness Center,Vegetarian / Vegan Restaurant,Sandwich Place,Hotel,Farm,Ice Cream Shop,Italian Restaurant,Lounge
4,Jogeshwari,Coffee Shop,Indian Restaurant,Pizza Place,Smoke Shop,Pharmacy,Restaurant,Sandwich Place,Chinese Restaurant,Mughlai Restaurant,Fast Food Restaurant


In [202]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 10

mumbai_grouped_clustering = mumbai_grouped.drop(columns=['Neighbourhood'])

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 5, 7, 9, 6, 0, 8, 5, 4, 3], dtype=int32)

In [203]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [204]:
neighbourhoods_venues_sorted

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Andheri,Indian Restaurant,Fast Food Restaurant,Bakery,Ice Cream Shop,Café,Sandwich Place,Vegetarian / Vegan Restaurant,Food Court,Department Store,Pizza Place
1,5,Borivali,Ice Cream Shop,Chinese Restaurant,Food Truck,Indian Restaurant,Clothing Store,Sandwich Place,Gym,Gym / Fitness Center,Fast Food Restaurant,Juice Bar
2,7,Dahisar,Indian Restaurant,Chinese Restaurant,Restaurant,Juice Bar,Café,Bar,Seafood Restaurant,Diner,Gym,Hotel Bar
3,9,Goregaon,Fast Food Restaurant,Indian Restaurant,Gym / Fitness Center,Vegetarian / Vegan Restaurant,Sandwich Place,Hotel,Farm,Ice Cream Shop,Italian Restaurant,Lounge
4,6,Jogeshwari,Coffee Shop,Indian Restaurant,Pizza Place,Smoke Shop,Pharmacy,Restaurant,Sandwich Place,Chinese Restaurant,Mughlai Restaurant,Fast Food Restaurant
5,0,Juhu,Hotel,Asian Restaurant,Indian Restaurant,Spa,Lounge,Cocktail Bar,Bar,Seafood Restaurant,Café,American Restaurant
6,8,Kandivali east,Pizza Place,Fast Food Restaurant,Indian Restaurant,Lounge,Café,Restaurant,Ice Cream Shop,Indie Movie Theater,Department Store,Multiplex
7,5,Kandivali west,Ice Cream Shop,Electronics Store,Pizza Place,Gym / Fitness Center,Fast Food Restaurant,Indian Restaurant,Café,Coffee Shop,Restaurant,Food
8,4,Malad,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Dessert Shop,Department Store,Pizza Place,Indian Restaurant,Garden,Ice Cream Shop,Food Truck
9,3,Mira-Bhayandar,Gym,Indian Restaurant,Fast Food Restaurant,Pizza Place,Shop & Service,Garden,Electronics Store,Mexican Restaurant,Diner,Restaurant


In [206]:
mumbai_merged = df_mumbai

mumbai_merged = mumbai_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

mumbai_merged.head() 

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Andheri,19.119698,72.846420,2.0,Indian Restaurant,Fast Food Restaurant,Bakery,Ice Cream Shop,Café,Sandwich Place,Vegetarian / Vegan Restaurant,Food Court,Department Store,Pizza Place
2,Mira-Bhayandar,19.295543,72.857359,3.0,Gym,Indian Restaurant,Fast Food Restaurant,Pizza Place,Shop & Service,Garden,Electronics Store,Mexican Restaurant,Diner,Restaurant
3,Bandra,26.049014,85.622828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Borivali,19.229068,72.857363,5.0,Ice Cream Shop,Chinese Restaurant,Food Truck,Indian Restaurant,Clothing Store,Sandwich Place,Gym,Gym / Fitness Center,Fast Food Restaurant,Juice Bar
5,Dahisar,19.249450,72.859621,7.0,Indian Restaurant,Chinese Restaurant,Restaurant,Juice Bar,Café,Bar,Seafood Restaurant,Diner,Gym,Hotel Bar


In [208]:
mumbai_merged=mumbai_merged.dropna()

In [209]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Neighbourhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Analysis

In [219]:
#for cluster 1
i=1
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == (i - 1), mumbai_merged.columns[[0] + np.arange(4, mumbai_merged.shape[1]).tolist()]]
    


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Juhu,Hotel,Asian Restaurant,Indian Restaurant,Spa,Lounge,Cocktail Bar,Bar,Seafood Restaurant,Café,American Restaurant


In [220]:
#for cluster 2
i=2
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == (i - 1), mumbai_merged.columns[[0] + np.arange(4, mumbai_merged.shape[1]).tolist()]]
    


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Vasai,Diner,Indian Restaurant,Snack Place,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Dessert Shop,Donut Shop,Electronics Store,Falafel Restaurant,Farm


In [221]:
#for cluster 3
i=3
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == (i - 1), mumbai_merged.columns[[0] + np.arange(4, mumbai_merged.shape[1]).tolist()]]
    


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Andheri,Indian Restaurant,Fast Food Restaurant,Bakery,Ice Cream Shop,Café,Sandwich Place,Vegetarian / Vegan Restaurant,Food Court,Department Store,Pizza Place
15,Virar,Indian Restaurant,Juice Bar,Bakery,Ice Cream Shop,Platform,Sandwich Place,Chinese Restaurant,Café,Diner,Indian Chinese Restaurant


In [222]:
#for cluster 4
i=4
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == (i - 1), mumbai_merged.columns[[0] + np.arange(4, mumbai_merged.shape[1]).tolist()]]
    


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Mira-Bhayandar,Gym,Indian Restaurant,Fast Food Restaurant,Pizza Place,Shop & Service,Garden,Electronics Store,Mexican Restaurant,Diner,Restaurant


In [223]:
#for cluster 5
i=5
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == (i - 1), mumbai_merged.columns[[0] + np.arange(4, mumbai_merged.shape[1]).tolist()]]
    


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Malad,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Dessert Shop,Department Store,Pizza Place,Indian Restaurant,Garden,Ice Cream Shop,Food Truck


In [224]:
#for cluster 6
i=6
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == (i - 1), mumbai_merged.columns[[0] + np.arange(4, mumbai_merged.shape[1]).tolist()]]
    


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Borivali,Ice Cream Shop,Chinese Restaurant,Food Truck,Indian Restaurant,Clothing Store,Sandwich Place,Gym,Gym / Fitness Center,Fast Food Restaurant,Juice Bar
9,Kandivali west,Ice Cream Shop,Electronics Store,Pizza Place,Gym / Fitness Center,Fast Food Restaurant,Indian Restaurant,Café,Coffee Shop,Restaurant,Food


In [225]:
#for cluster 7
i=7
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == (i - 1), mumbai_merged.columns[[0] + np.arange(4, mumbai_merged.shape[1]).tolist()]]
    


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Jogeshwari,Coffee Shop,Indian Restaurant,Pizza Place,Smoke Shop,Pharmacy,Restaurant,Sandwich Place,Chinese Restaurant,Mughlai Restaurant,Fast Food Restaurant


In [226]:
#for cluster 8
i=8
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == (i - 1), mumbai_merged.columns[[0] + np.arange(4, mumbai_merged.shape[1]).tolist()]]
    


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Dahisar,Indian Restaurant,Chinese Restaurant,Restaurant,Juice Bar,Café,Bar,Seafood Restaurant,Diner,Gym,Hotel Bar


In [227]:
#for cluster 9
i=9
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == (i - 1), mumbai_merged.columns[[0] + np.arange(4, mumbai_merged.shape[1]).tolist()]]
    


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Kandivali east,Pizza Place,Fast Food Restaurant,Indian Restaurant,Lounge,Café,Restaurant,Ice Cream Shop,Indie Movie Theater,Department Store,Multiplex


In [228]:
#for cluster 10
i=10
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == (i - 1), mumbai_merged.columns[[0] + np.arange(4, mumbai_merged.shape[1]).tolist()]]
    


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Goregaon,Fast Food Restaurant,Indian Restaurant,Gym / Fitness Center,Vegetarian / Vegan Restaurant,Sandwich Place,Hotel,Farm,Ice Cream Shop,Italian Restaurant,Lounge


From above data it is clear that in cluster 7 jogeswari has fastfood restaurant at 10th common place so we may try there.